In [1]:
import torch
torch.cuda.empty_cache()
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
# Ensure pad_token_id is set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    # attn_implementation="flash_attention_2"
).cuda()
# model.half()

In [1]:
print("xd")

xd


In [8]:
# Initialize model with desired settings

# model.half()
# model.bfloat16()
# model.resize_token_embeddings(len(tokenizer))


#training .. 

training_args = TrainingArguments(output_dir='./results',
                                  # num_train_epochs=1,
                                  # load_best_model_at_end=True,
                                  max_steps=10, 
                                  logging_steps=1,
                                  overwrite_output_dir=True,
                                  save_steps=10,
                                  per_device_train_batch_size=20,
                                  per_device_eval_batch_size=20,
                                  warmup_steps=10,
                                  weight_decay=0.05,
                                  logging_dir='./logs',
                                  report_to = 'none')

trainer = Trainer(model=model,
        args=training_args,
        train_dataset = train_dataset, 
        eval_dataset = val_dataset,
        data_collator = lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                      'attention_mask': torch.stack([f[1] for f in data]),
                                      'labels': torch.stack([f[0] for f in data])})

trainer.train()
# model.save_pretrained(output_path)
# tokenizer.save_pretrained(output_path)



# Prepare the input text
# input_text = "<|startoftext|> Title: Sharptalon's Claw Description: "
input_text = "Once upon a time "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.cuda()

# Ensure the model is in eval mode
model.eval()

# Generate text
try:
    sample_outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        top_k=50,
        max_length=300,
        top_p=0.95,
        temperature=0.7,
        num_return_sequences=20
    )
    # Decode and print generated texts
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in sample_outputs]
    for i, text in enumerate(generated_texts):
        print(f"Generated text {i+1}:\n{text}\n")

except RuntimeError as e:
    print("RuntimeError during generation:", e)

    # Additional Debugging: Check logits
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        logits = outputs.logits
        assert not torch.isnan(logits).any(), "logits contain NaNs"
        assert not torch.isinf(logits).any(), "logits contain Infs"
        print("Logits sample:", logits[0, -1, :10])


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [12]:
model.eval()
input_text = "<|startoftext|> Title: Sharptalon's Claw Description: "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.cuda()

# Generate text
try:
    sample_outputs = model.generate(
        input_ids=input_ids,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        top_k=50,
        max_length=300,
        top_p=0.95,
        temperature=0.7,
        num_return_sequences=20
    )
    # Decode and print generated texts
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in sample_outputs]
    for i, text in enumerate(generated_texts):
        print(f"Generated text {i+1}:\n{text}\n")

except RuntimeError as e:
    print("RuntimeError during generation:", e)

    # Additional Debugging: Check logits
    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        logits = outputs.logits
        assert not torch.isnan(logits).any(), "logits contain NaNs"
        assert not torch.isinf(logits).any(), "logits contain Infs"
        print("Logits sample:", logits[0, -1, :10])

Generated text 1:
 Title: Sharptalon's Claw Description: Â· Sharpt of The is A- (.
 is

 is

,
 by_
[SÂ± Juventus in U B the to youtu a U  a key U a K U U   C U  U
 "  M  hairc  hairc
 

Generated text 2:
 Title: Sharptalon's Claw Description: ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [1]:
import gc
import torch
import pandas as pd
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, GPT2LMHeadModel, GPTNeoForCausalLM, AutoModelForCausalLM
# torch.manual_seed(42)

### Loading GPT2-Medium Model from 🤗 Model Hub 

In [7]:
import os
model_id = "EleutherAI/gpt-neo-1.3B"
output_path = os.path.join("Magisterka", "EleutherAI/gpt-neo-1.3B")

try:
    model = AutoModelForCausalLM.from_pretrained(output_path, torch_dtype=torch.float16, attn_implementation="flash_attention_2").cuda() #GPT2LMHeadModel||GPTNeoForCausalLM
    tokenizer = AutoTokenizer.from_pretrained(output_path, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
except:
    tokenizer = AutoTokenizer.from_pretrained(model_id, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, attn_implementation="flash_attention_2").cuda()
    # model = GPT2LMHeadModel.from_pretrained(model_id) #model_id
model.to('cuda')
model.resize_token_embeddings(len(tokenizer))

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Embedding(50259, 2048)

In [2]:
texts = pd.read_csv('data.csv')
texts = texts.dropna()
texts = texts.dropna(subset=['Quest Description'])
# print(texts)
# print(texts['Quest Description'])

In [3]:
max_length = max([len(tokenizer.encode('Title: ' + title + 'Description: ' + descr)) for title, descr in zip(texts['Quest Description'], texts['Quest Title'])])
max_length2 = max([len(tokenizer.encode('<|startoftext|>' + 'Title: ' + title + 'Description: ' + descr + '<|endoftext|>')) for title, descr in zip(texts['Quest Description'], texts['Quest Title'])])
print(max_length)
print(max_length2)

NameError: name 'tokenizer' is not defined

In [ ]:
class TextDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.labels = []
        self.input_ids = []
        self.attn_masks = []        
        for title, descrp in zip(txt_list['Quest Title'], txt_list['Quest Description']):
            encodings_dict = tokenizer('<|startoftext|>' + 'Title: ' + title + 'Description: ' + descrp + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    def __len__(self): return len(self.input_ids)
    def __getitem__(self, idx): return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
dataset = TextDataset(texts, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
gc.collect()
print(train_size)

In [1]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [9]:
training_args = TrainingArguments(output_dir='./results',
                                  num_train_epochs=1,
                                  logging_steps=1,
                                  overwrite_output_dir=True,
                                  save_steps=100,
                                  per_device_train_batch_size=20,
                                  per_device_eval_batch_size=20,
                                  warmup_steps=10,
                                  weight_decay=0.05,
                                  logging_dir='./logs',
                                  load_best_model_at_end=True,
                                  report_to = 'none')

In [10]:
trainer = Trainer(model=model,
        args=training_args,
        train_dataset = train_dataset, 
        eval_dataset = val_dataset,
        data_collator = lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                      'attention_mask': torch.stack([f[1] for f in data]),
                                      'labels': torch.stack([f[0] for f in data])})

In [11]:
# trainer.train()
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)

Step,Training Loss
10,3.345100
20,0.000000
30,0.000000
40,0.000000
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


TrainOutput(global_step=305, training_loss=0.10967597336065574, metrics={'train_runtime': 592.4746, 'train_samples_per_second': 10.289, 'train_steps_per_second': 0.515, 'total_flos': 7646678926884864.0, 'train_loss': 0.10967597336065574, 'epoch': 1.0})

### GPT Generated Texts

In [25]:
input_ids = tokenizer("<|startoftext|> Title: Sharptalon's Claw Description: ", return_tensors="pt").input_ids.cuda()
input_ids2 = tokenizer("<|startoftext|> Title: Iskaara Tuskarr Reputation Description: ", return_tensors="pt").input_ids.cuda()
# Create attention mask
attention_mask = torch.ones(input_ids.shape, dtype=torch.long).cuda()

# Set pad_token_id to eos_token_id if it's not set
pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

assert not torch.isnan(input_ids).any(), "input_ids contain NaNs"
assert not torch.isinf(input_ids).any(), "input_ids contain Infs"
assert not torch.isnan(attention_mask).any(), "attention_mask contains NaNs"
assert not torch.isinf(attention_mask).any(), "attention_mask contains Infs"
model_inputs = tokenizer(["<|startoftext|> Title: Sharptalon's Claw Description: "], return_tensors="pt").to("cuda:0")
print(model_inputs)

{'input_ids': tensor([[50257, 11851,    25, 15465,   457, 40755,   338, 30358, 12489,    25,
           220]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [32]:
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "left"
model = model.bfloat16()
model = model.half().cuda()
sample_outputs = model.generate(**model_inputs,
                                pad_token_id=pad_token_id,
                                do_sample = True,
                                top_k = 50, 
                                max_length = 300,
                                top_p = 0.95,
                                temperature = 0.7,
                                num_return_sequences = 20)

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0

In [ ]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

### Original Texts (Random)

In [ ]:
pd.options.display.max_colwidth = 1000
texts.sample(10)